In [1]:
%cd /ibex/user/slimhy/PADS/code/
import glob
import os
import numpy as np
from util.misc import load_pickle


DATA_PATH = "/ibex/project/c2273/3DCoMPaT/packaged"
OUT_PATH =  "/ibex/project/c2273/3DCoMPaT/bounding_boxes"

def radial_sort(points):
    """
    Radially sort points in a NumPy array.
    
    :param points: NumPy array of shape (n, 2) where n is the number of points
    :return: NumPy array of the same shape, with points sorted radially
    """
    # Calculate the centroid
    centroid = np.mean(points, axis=0)
    
    # Calculate the angles
    angles = np.arctan2(points[:, 1] - centroid[1], points[:, 0] - centroid[0])
    
    # Sort the points based on their angles
    sorted_indices = np.argsort(angles)
    
    return points[sorted_indices]


def dump_mat(mat, f_name):
    """
    Dump a numpy matrix to a npy file.
    """
    f_out = os.path.join(OUT_PATH, f_name)
    np.save(f_out, mat)
    
    
def get_part_labels(bb):
    return np.array([b[0][1] for b in bb])


def get_part_bbs(bb):
    return np.array([radial_sort(np.array(b[1].vertices)) for b in bb])

In [2]:
from tqdm.notebook import tqdm

all_files = glob.glob(os.path.join(DATA_PATH, "*bbs.pkl"))

for f_name in tqdm(all_files):
    bb = load_pickle(f_name)
    part_labels = get_part_labels(bb)
    break

In [24]:
file_list = []
latents_dir = os.path.join("/ibex/project/c2273/PADS/3DCoMPaT", "latents")
bbs_dir = os.path.join("/ibex/project/c2273/PADS/3DCoMPaT", "bounding_boxes")
PART_CAP = 24
for f in sorted(os.listdir(latents_dir)):
    if f.endswith(".npy"):
        file_type = "_".join(f.split("_")[2:-1])
        latent_f = f
        bb_coords_f = f.replace(".npy", "_part_bbs.npy")
        bb_coords = np.load(os.path.join(bbs_dir, bb_coords_f))
        if bb_coords.shape[0] > PART_CAP:
            continue
        file_list += [latent_f.replace(".npy", "")]
    # break

In [25]:
file_list.sort()
# Dump to JSON
import json

with open(os.path.join("/ibex/project/c2273/PADS/3DCoMPaT", "file_list_capped.json"), "w") as f:
    json.dump(file_list, f)

In [26]:
len(file_list)

In [27]:
final_list = []
for f in file_list:
    file_type = "_".join(f.split("_")[2:-1])
    if file_type not in []:
        bb_coords_f = f + "_part_bbs"
        bb_labels_f = f + "_part_labels"
        final_list += [[k + ".npy" for k in [f, bb_coords_f, bb_labels_f]]]


In [ ]:
float(-6.2343e-02)

In [28]:
final_list